# Auto check

In [ ]:

"""
EQB = [ [EQB item] * number of EQB item] 
[EQB item] = [ [paragraph1 in ITB1] , [ paragraph2 in ITB2] , [paragraph3 in ITB3] ] 

this is a autocheck
"""

#only use EOEG,NC6,JGS,liwa as training data
#use PTT_LNG as valid data 


import pandas as pd
import numpy as np
import nltk
import math
import re

In [2]:
pd_EOEG = pd.read_excel('EQB.xlsx',sheet_name = 'EOEG' );
pd_NC6 = pd.read_excel("EQB.xlsx",sheet_name='NC6');
pd_JGS = pd.read_excel("EQB.xlsx",sheet_name= 'JGS' );
pd_liwa = pd.read_excel("EQB.xlsx",sheet_name= 'LIWA' );
pd_PTT = pd.read_excel("EQB.xlsx",sheet_name= 'PTT LNG' );

In [3]:
EQB = [ [] for i in range(406) ];

def add_EQB(df):
    check = df['paragraph'].isna();
    for i in range(len(df)): 
        if check[i] == False:
            EQB[i].append(df['paragraph'][i])

add_EQB(pd_NC6);
add_EQB(pd_EOEG);
add_EQB(pd_JGS);
add_EQB(pd_liwa);



In [4]:
from nltk.corpus import stopwords

from nltk.stem import SnowballStemmer
import  nltk;
import math;
from nltk.stem import PorterStemmer
stopword_list = list(set(stopwords.words('english')));
stopword_list.append(',');
stopword_list.append(')');
stopword_list.append('(');
stopword_list.append('.');
stopword_list.append('-');

stemmer = SnowballStemmer('english')


In [5]:
def preprocess_paragraph(paragraph):
    raw_words = nltk.word_tokenize(paragraph); #tokenize words by nltk
    words = [w for w in raw_words if not w in stopword_list] #  drop stopword
    words = [stemmer.stem(w) for w in words]
    
    drop_digit = []
    for w in words:
        if re.findall(r'^[^\d]\w+',w):
            after = re.findall(r'^\w+$',w);
            if len(after) > 0:
                drop_digit.append(after[0])
            
    return drop_digit;

In [6]:
def update_dic(key , dic): #dictnary中令一个key的value+1
    if key not in dic:
        dic[key] = 1;
    else: 
        dic[key] += 1;
    return dic

In [7]:
def get_tf(EQB_field , df):
    num_words = 0;
    freq = {};
    for paragraph in EQB_field:
        words = preprocess_paragraph(paragraph)
        num_words += len(words) # calculate the total number of these paragraphs
        for w in words:
            freq = update_dic( key = w , dic = freq) # calculate tf
    tf = {}        
    for (word , count) in freq.items():
        tf[word] = count * 1.0 / num_words; # 修正每个词为正确的频率
        df = update_dic( key = word , dic = df)    #calculate df
    return tf , df;

In [8]:
def get_tfidf(tf_list , df , N):
    tfidf_list = []
    for tf in tf_list:
        tfidf = {}
        for (word , tf_value) in tf.items():
            tfidf[word] = tf_value * math.log( N / ( 1 + df[word]) , 2 ) 
        tfidf_list.append(tfidf);
    return tfidf_list

In [12]:
def valid(word,paragraph):
    f = re.findall(word , paragraph ,re.IGNORECASE);
    if len(f) > 0 :
        return 1 ;
    else:
        return 0;

In [13]:
def solve(EQB):
    df = {}
    tf_list = []
    id_list = []
    N = len(EQB);
    
    paragraph_num = []
    EQB_item_id = [];
    item = pd_EOEG['Item'][0] 
    for i in range(len(EQB)):
        EQB_field = EQB[i];
        id = pd_EOEG['Quality Activity'][i] 
        if (pd_EOEG['Item'].isna()[i] == False):
            item = pd_EOEG['Item'][i] 
        
        if len(EQB_field) == 0:
            N = N - 1
            continue    
        tf , df = get_tf(EQB_field, df)
        tf_list.append(tf)
        paragraph_num.append(len(EQB_field))
        id_list.append(item + ' ' + id)
        EQB_item_id.append(i)
        
    tfidf_list = get_tfidf(tf_list , df , N)
    
    tot = 0
    for i in range(len(tfidf_list)):
        if pd_PTT['paragraph'].isna()[EQB_item_id[i]] == True:
            continue
        tfidf = tfidf_list[i]
        id = id_list[i]
        tot+=1;
        print(id)
        print('come form',paragraph_num[i],'ITB')
        s = pd.Series(tfidf)
        top10 = s.sort_values(ascending=False).head(10)
        v = []
        df = pd.DataFrame( {'keyword':top10.index , 'tf-idf':top10.data} )
        hits = [];
        for j in range(len(top10)):
            v = valid(top10.index[j] , pd_PTT['paragraph'][EQB_item_id[i]])
            hits.append(v)
        
        df['hits'] = hits;
        print(df)


In [14]:
solve(EQB)

1.1 a. Minimun Vertical Clearance for Access
come form 4 ITB
    keyword    tf-idf  hits
0   walkway  0.453391     1
1  headroom  0.372746     1
2  platform  0.287234     0
3  clearanc  0.253007     1
4      work  0.181357     0
5  overhead  0.149098     0
6      elev  0.149098     0
7       ead  0.106807     1
8  stairway  0.106807     0
9   whichev  0.097372     0
1.1 b. Minimun width Clearance for Access
come form 3 ITB
    keyword    tf-idf  hits
0  platform  0.445213     0
1   walkway  0.374803     1
2     width  0.308137     0
3  clearanc  0.261441     1
4   minimum  0.249277     0
5    passag  0.220735     0
6      aisl  0.201236     0
7     escap  0.201236     0
8    arrang  0.167903     0
9  horizont  0.160490     1
1.2 a. Maximum distance between landings on a sectional ladder
come form 4 ITB
    keyword    tf-idf  hits
0  platform  0.455331     1
1    ladder  0.411620     0
2   distanc  0.202369     1
3   maximum  0.183504     1
4       run  0.171719     0
5    vertic  0.171

/home/hzk/.local/lib/python3.6/site-packages/ipykernel_launcher.py:39: FutureWarning: Series.data is deprecated and will be removed in a future version


       keyword    tf-idf  hits
0         valv  0.552516     1
1        block  0.409294     1
2         gate  0.269498     0
3         isol  0.269498     0
4        fulli  0.204647     0
5        indic  0.204647     0
6         main  0.179725     0
7        posit  0.156680     0
8  underground  0.156680     0
9         ring  0.156680     0
5.9.1 Type of foam concentrate for hydrocarbon fire
come form 2 ITB
   keyword    tf-idf  hits
0     foam  0.535194     1
1  aqueous  0.189201     0
2     afff  0.189201     0
3     film  0.189201     0
4       fp  0.189201     0
5     form  0.160630     0
6   applic  0.137563     0
7  protect  0.103487     0
8     tank  0.098632     0
9    treat  0.094601     0
5.10.1 a. Agent
come form 2 ITB
      keyword    tf-idf  hits
0  extinguish  0.396504     1
1     portabl  0.217508     1
2          kg  0.187063     1
3      chemic  0.177780     1
4        fire  0.175428     0
5         dri  0.146916     0
6       small  0.142049     0
7   potassium  0.13228